# Dask Array

Материалы:
* Макрушин С.В. Лекция 11: Dask
* https://docs.dask.org/en/latest/array.html
* JESSE C. DANIEL. Data Science with Python and Dask. 

## Задачи для совместного разбора

1. Создайте массив размерностью 1000 на 300000, заполненный числами из стандартного нормального распределения. Исследуйте основные характеристики полученного массива.

In [1]:
import dask.array as da
import h5py
import numpy as np

In [2]:
import dask

dask.__version__

'2.30.0'

In [3]:
%%time
arr_np = np.random.normal(0,1, size=(1_000, 300_000))

CPU times: user 9.27 s, sys: 1.03 s, total: 10.3 s
Wall time: 11.3 s


In [4]:
%%time
arr_da = da.random.normal(0,1, size=(1_000, 300_000), 
                          chunks=(1_000, 30_000))

CPU times: user 3.43 ms, sys: 5.54 ms, total: 8.97 ms
Wall time: 12.6 ms


In [5]:
arr_da

dask.array<normal, shape=(1000, 300000), dtype=float64, chunksize=(1000, 30000), chunktype=numpy.ndarray>

2. Посчитайте сумму квадратов элементов массива, созданного в задаче 1. Создайте массив `np.array` такого же размера и сравните скорость решения задачи с использование `da.array` и `np.array`

In [6]:
%%time
np.power(arr_da, 2).sum()

CPU times: user 7.56 ms, sys: 3.29 ms, total: 10.8 ms
Wall time: 17.1 ms


dask.array<sum-aggregate, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>

3. Визуализируйте граф вычислений для задачи 2.

## Лабораторная работа 11

In [7]:
import dask.array as da
import h5py
import numpy as np

1. Считайте датасет `recipe` из файла `minutes_n_ingredients_full.hdf5` в виде `dask.array`. Укажите аргумент `chunks=(100_000, 3)` при создании массива. Выведите на экран основную информацию о массиве.

In [8]:
with h5py.File('./data/minutes_n_ingredients_full.hdf5', 'r') as h5f: 
    for key in h5f.keys():
        print(f'key => {key}')
        vals = [val for val in h5f[key].attrs.values()]
        print(vals)
        
# чтения комментария датасета, нужно для дальнейшней работы

key => recipe
['Содержит столбцы id, minutes и n_ingredients из recipes_full.csv']


In [9]:
f = h5py.File('./data/minutes_n_ingredients_full.hdf5', 'r')
d = f['/recipe']
recipe = da.from_array(d, chunks=(100_000, 3))

In [10]:
recipe

dask.array<array, shape=(2231637, 3), dtype=int64, chunksize=(100000, 3), chunktype=numpy.ndarray>

In [11]:
recipe.compute()

array([[ 683970,      33,       9],
       [1089012,      23,       5],
       [1428572,       0,       5],
       ...,
       [1910650,      60,       2],
       [ 713836,       0,       9],
       [ 660699,      64,       8]])

2. Вычислите среднее значение по каждому столбцу, кроме первого. 

In [12]:
res = da.mean(recipe[:, 1:], axis=0).compute()

In [13]:
res

array([1004.20805176,    5.4198008 ])

3. Исследуйте, как влияет значение аргумента `chunks` при создании `dask.array` на скорость выполнения операции поиска среднего. 

In [14]:
with h5py.File('./data/minutes_n_ingredients_full.hdf5', 'r') as f:
    d = f['recipe']
    recipe_3_1 = da.from_array(d)
    recipe_3_2 = da.from_array(d, chunks=(50_000, 3))
    recipe_3_3 = da.from_array(d, chunks=(300_000, 3))
    recipe_3_4 = da.from_array(d, chunks=(300_000, 1))
    recipe_3_5 = da.from_array(d, chunks=(800_000, 3))
    recipe_3_6 = da.from_array(d, chunks=(2_000_000, 1))
    recipe_3_7 = da.from_array(d, chunks=(2_000_000, 3))

In [15]:
recipe_3_1 

dask.array<array, shape=(2231637, 3), dtype=int64, chunksize=(2231637, 3), chunktype=numpy.ndarray>

In [16]:
recipe_3_2

dask.array<array, shape=(2231637, 3), dtype=int64, chunksize=(50000, 3), chunktype=numpy.ndarray>

In [17]:
recipe_3_3

dask.array<array, shape=(2231637, 3), dtype=int64, chunksize=(300000, 3), chunktype=numpy.ndarray>

In [18]:
recipe_3_4 

dask.array<array, shape=(2231637, 3), dtype=int64, chunksize=(300000, 1), chunktype=numpy.ndarray>

In [19]:
recipe_3_5 

dask.array<array, shape=(2231637, 3), dtype=int64, chunksize=(800000, 3), chunktype=numpy.ndarray>

In [20]:
recipe_3_6 

dask.array<array, shape=(2231637, 3), dtype=int64, chunksize=(2000000, 1), chunktype=numpy.ndarray>

In [21]:
recipe_3_7

dask.array<array, shape=(2231637, 3), dtype=int64, chunksize=(2000000, 3), chunktype=numpy.ndarray>

In [22]:
%%time
da.mean(recipe_3_1[:, 1:], axis=0)

CPU times: user 1.76 ms, sys: 44 µs, total: 1.81 ms
Wall time: 1.79 ms


dask.array<mean_agg-aggregate, shape=(2,), dtype=float64, chunksize=(2,), chunktype=numpy.ndarray>

In [23]:
%%time
# chunks=(10_000, 3)
da.mean(recipe_3_2[:, 1:], axis=0)

CPU times: user 3.56 ms, sys: 149 µs, total: 3.71 ms
Wall time: 6.49 ms


dask.array<mean_agg-aggregate, shape=(2,), dtype=float64, chunksize=(2,), chunktype=numpy.ndarray>

In [24]:
%%time
# chunks=(300_000, 3)
da.mean(recipe_3_3[:, 1:], axis=0)

CPU times: user 2.21 ms, sys: 74 µs, total: 2.28 ms
Wall time: 2.25 ms


dask.array<mean_agg-aggregate, shape=(2,), dtype=float64, chunksize=(2,), chunktype=numpy.ndarray>

In [25]:
%%time
# chunks=(300_000, 1)
da.mean(recipe_3_4[:, 1:], axis=0)

CPU times: user 2.19 ms, sys: 78 µs, total: 2.26 ms
Wall time: 2.23 ms


dask.array<mean_agg-aggregate, shape=(2,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>

In [26]:
%%time
# chunks=(800_000, 3)
da.mean(recipe_3_5[:, 1:], axis=0)

CPU times: user 3.27 ms, sys: 141 µs, total: 3.41 ms
Wall time: 3.49 ms


dask.array<mean_agg-aggregate, shape=(2,), dtype=float64, chunksize=(2,), chunktype=numpy.ndarray>

In [27]:
%%time
# chunks=(2_231_636, 1)
da.mean(recipe_3_6[:, 1:], axis=0)

CPU times: user 2.26 ms, sys: 95 µs, total: 2.35 ms
Wall time: 2.45 ms


dask.array<mean_agg-aggregate, shape=(2,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>

In [28]:
%%time
# chunks=(2_231_636, 3)
da.mean(recipe_3_7[:, 1:], axis=0)

CPU times: user 2.89 ms, sys: 550 µs, total: 3.44 ms
Wall time: 3.16 ms


dask.array<mean_agg-aggregate, shape=(2,), dtype=float64, chunksize=(2,), chunktype=numpy.ndarray>

Мало "кусочков" - плохо, но много "кусочков" - тоже плохо. Придерживаемся золотой середины

4. Выберите рецепты, время выполнения которых меньше медианного значения

In [29]:
recipe[recipe[:,1] < da.median(recipe[:,1], axis=0)].compute()

array([[1089012,      23,       5],
       [1428572,       0,       5],
       [1400250,      24,       1],
       ...,
       [1029131,      19,       4],
       [1700703,       1,       1],
       [ 713836,       0,       9]])

5. Посчитайте количество каждого из возможных значений кол-ва ингредиентов

In [30]:
import dask.bag as db

In [31]:
res_5 = da.unique(recipe[:,2], return_counts=True)
list(zip(res_5[0].compute(), res_5[1].compute()))

[(1, 222071),
 (2, 224158),
 (3, 229388),
 (4, 234948),
 (5, 240720),
 (6, 244360),
 (7, 247181),
 (8, 246747),
 (9, 246816),
 (10, 22430),
 (11, 19094),
 (12, 15165),
 (13, 11640),
 (14, 8284),
 (15, 6014),
 (16, 4145),
 (17, 2793),
 (18, 1913),
 (19, 1279),
 (20, 852),
 (21, 529),
 (22, 346),
 (23, 244),
 (24, 178),
 (25, 107),
 (26, 68),
 (27, 55),
 (28, 33),
 (29, 22),
 (30, 20),
 (31, 13),
 (32, 5),
 (33, 4),
 (34, 3),
 (35, 4),
 (36, 1),
 (37, 2),
 (38, 1),
 (39, 1),
 (40, 2),
 (43, 1)]

In [32]:
res_5[1].sum().compute()

2231637

In [33]:
# v2
tuples = da.unique(recipe[:,2], return_counts=True)
lst = [i.compute() for i in tuples]
da.dstack(lst).compute()

array([[[     1, 222071],
        [     2, 224158],
        [     3, 229388],
        [     4, 234948],
        [     5, 240720],
        [     6, 244360],
        [     7, 247181],
        [     8, 246747],
        [     9, 246816],
        [    10,  22430],
        [    11,  19094],
        [    12,  15165],
        [    13,  11640],
        [    14,   8284],
        [    15,   6014],
        [    16,   4145],
        [    17,   2793],
        [    18,   1913],
        [    19,   1279],
        [    20,    852],
        [    21,    529],
        [    22,    346],
        [    23,    244],
        [    24,    178],
        [    25,    107],
        [    26,     68],
        [    27,     55],
        [    28,     33],
        [    29,     22],
        [    30,     20],
        [    31,     13],
        [    32,      5],
        [    33,      4],
        [    34,      3],
        [    35,      4],
        [    36,      1],
        [    37,      2],
        [    38,      1],
        [   

6. Найдите максимальную продолжительность рецепта. Ограничьте максимальную продолжительность рецептов сверху значением, равному 75% квантилю.

In [36]:
da.percentile(recipe[:,1], 75).compute()

array([49.])

In [37]:
# включать или не включать, вот в чем вопрос...
da.max(recipe[recipe[:,1] < da.percentile(recipe[:,1], 75).compute()][:,1]).compute()

48

7. Создайте массив `dask.array` из 2 чисел, содержащих ваши предпочтения относительно времени выполнения рецепта и кол-ва ингредиентов. Найдите наиболее похожий (в смысле $L_1$) рецепт из имеющихся в датасете.

In [38]:
# точное совпадение
array_7 = da.from_array([45,25])
r7 = (np.abs(recipe[:,1] - array_7[0]) + np.abs(recipe[:,2] - array_7[1])).argmin().compute()
recipe[r7].compute()

array([57284,    45,    25])

In [39]:
# неточное совпадение: v1
array_7_1 = da.from_array([45,30])
r7_1 = (np.abs(recipe[:,1] - array_7_1[0]) + np.abs(recipe[:,2] - array_7_1[1])).argmin().compute()
recipe[r7_1].compute()

array([350600,     45,     29])

In [40]:
# неточное совпадение: v2
array_7_2 = da.from_array([45,31])
r7_2 = (np.abs(recipe[:,1] - array_7_2[0]) + np.abs(recipe[:,2] - array_7_2[1])).argmin().compute()
recipe[r7_2].compute()

array([453970,     45,     32])

In [41]:
arr = (recipe[:,1:] - array_7_1)
arr.compute()

array([[-12, -21],
       [-22, -25],
       [-45, -25],
       ...,
       [ 15, -28],
       [-45, -21],
       [ 19, -22]])

8. Работая с исходным файлом в формате `hdf5`, реализуйте алгоритм подсчета среднего значения в блочной форме и вычислите с его помощью среднее значение второго столбца в массиве.

Блочный алгоритм вычислений состоит из двух частей:
1. загрузка фрагмента за фрагментом данных по `blocksize` элементов и проведение вычислений на этим фрагментом;
2. агрегация результатов вычислений на различных фрагментах для получения результата на уровне всего набора данных.

Важно: при работе с `h5py` в память загружаются не все элементы, а только те, которые запрашиваются в данный момент

In [42]:
recipe.numblocks[0]

23

In [43]:
recipe.shape[0]

2231637

In [44]:
with h5py.File('./data/minutes_n_ingredients_full.hdf5', 'r') as h5f: 
    d = h5f['/recipe']
    recipe = da.from_array(d, chunks=(100_000, 3))
    sums = 0
    for block_num in range(recipe.numblocks[0]):
        sums += da.sum(recipe.blocks[block_num][:,1])
    print((sums / recipe.shape[0]).compute())

1004.2080517575215
